In [3]:
# import torch 
from torch import nn
import torch
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.stats import skew
from scipy.stats import kurtosis
from prettytable import PrettyTable
import os
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Cuda Device Available")
    print("Name of the Cuda Device: ", torch.cuda.get_device_name())
    print("GPU Computational Capablity: ", torch.cuda.get_device_capability())

else:
    device = torch.device("cpu")
    
def changeCourtToValue(court):
    if court == 'Hard':
        return 1
    elif court == 'Clay':
        return 2
    elif court == 'Grass':
        return 3
    elif court == 'Carpet':
        return 4
    
def changeHandToValue(hand):
    if hand == 'R':
        return 1
    elif hand == 'U':
        return 0
    elif hand == 'L':
        return -1

    
    
def BuildDataset():
    df = pd.read_csv("data/atp_matches_2015.csv")
    for i in range(2016, 2023):
        newDf = pd.read_csv("data/atp_matches_" + str(i) + ".csv")
        df = pd.concat([df, newDf], join="inner", ignore_index = True)
    
    df = df[['surface', 'winner_hand', 'winner_ht', 'winner_age', 'winner_rank', 'winner_rank_points', 'loser_hand', 'loser_ht', 'loser_age', 'loser_rank', 'loser_rank_points']]
    
    
    avgSwitches = ['winner_ht', 'winner_age', 'loser_ht', 'loser_age']
    for i in avgSwitches:
        mean = df[i].mean()
        df[i] = df[i].fillna(mean)
        
    maxSwitches = ['winner_rank', 'loser_rank']
    for i in maxSwitches:
        Max = df[i].max()
        df[i] = df[i].fillna(Max)
    
    minSwitches = ['winner_rank_points', 'loser_rank_points']
    for i in minSwitches:
        df[i] = df[i].fillna(1)
        
    hands = ['winner_hand', 'loser_hand']
    for i in hands:
        df[i] = df[i].fillna('R')
    
    df['surface'] = df['surface'].apply(changeCourtToValue)
    df['winner_hand'] = df['winner_hand'].apply(changeHandToValue)
    df['loser_hand'] = df['loser_hand'].apply(changeHandToValue)
    df['Player1Wins'] = 1
    
    df.rename(columns = {'winner_hand': 'P1Hand', 'winner_ht': 'P1Height', 
                        'winner_age':'P1Age', 'winner_rank':'P1Rank', 'winner_rank_points': 'P1RankPoint',
                         'loser_hand': 'P2Hand', 'loser_ht': 'P2Height', 
                        'loser_age':'P2Age', 'loser_rank':'P2Rank', 'loser_rank_points': 'P2RankPoint'
                        }, inplace = True)
    
    for i in range(19622):
        if (i%2 == 0):
            df = swapPlayers(df, i)
    return df

def swapPlayers(df, index):
    df.loc[index, 'P1Hand'], df.loc[index, 'P2Hand'] = df.loc[index, 'P2Hand'], df.loc[index, 'P1Hand']
    df.loc[index, 'P1Height'], df.loc[index, 'P2Height'] = df.loc[index, 'P2Height'], df.loc[index, 'P1Height']
    df.loc[index, 'P1Age'], df.loc[index, 'P2Age'] = df.loc[index, 'P2Age'], df.loc[index, 'P1Age']
    df.loc[index, 'P1Rank'], df.loc[index, 'P2Rank'] = df.loc[index, 'P2Rank'], df.loc[index, 'P1Rank']
    df.loc[index, 'P1RankPoint'], df.loc[index, 'P2RankPoint'] = df.loc[index, 'P2RankPoint'], df.loc[index, 'P1RankPoint']
    df.loc[index, 'Player1Wins'] = 0
    return df

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.ff1 = nn.Linear(11, 32)
        self.ff2 = nn.Linear(32, 64)
        self.ff3 = nn.Linear(64, 32)
        self.ff4 = nn.Linear(32, 16)
        self.ff5 = nn.Linear(16, 4)
        self.ff6 = nn.Linear(4, 1)

    def forward(self, x):
        x = F.normalize(x)
        print(x, "\n")
        x = F.relu(self.ff1(x))
        print(x, "\n")
        x = F.relu(self.ff2(x))
        print(x, "\n")
        x = F.relu(self.ff3(x))
        print(x, "\n")
        x = F.relu(self.ff4(x))
        print(x, "\n")
        x = F.relu(self.ff5(x))
        print(x, "\n")
        x = torch.sigmoid(self.ff6(x))
        print(x, "\n")
        return x
    
def Train(model, X_train, y_train, epochs, learningRate, batchSize):
    
    criterion = nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)
    
    dataset = torch.utils.data.TensorDataset(X_train, y_train)
    dataLoader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batchSize, shuffle=True)
    
    for epoch in range(epochs):
        for i, (data, label) in enumerate(dataLoader):
         #   data = data
            label = label.type(torch.LongTensor)

            output = model(data)
            output = output.squeeze(1)
            output = output.float()
            label = label.float()
            print(output)
            print(label)
            output = torch.nan_to_num(output, nan=0.5)
            loss = criterion(output, label)
            print(loss)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            #Need Validation
            
        print("Epoch: " + str(epoch) + " Loss: " + str(loss.item()))

    print('Training Complete')
    
    

def main():
    df = BuildDataset()
    print(df.tail())
    
#main()
df = BuildDataset()
M = NeuralNet()
train, test = train_test_split(df, test_size=0.33)
X_train, y_train = torch.tensor(np.array(train.iloc[:, 0:-1])), torch.tensor(np.array(train.iloc[:, -1]))
X_test, y_test = torch.tensor(np.array(test.iloc[:, 0:-1])), torch.tensor(np.array(test.iloc[:, -1]))
X_train, y_train = X_train.float(), y_train.float()
print(X_train[0])
print(y_train[0])

#Train(M, X_train, y_train, 20, 0.001, 3)

tensor([1.0000e+00, 1.0000e+00, 1.8693e+02, 2.4345e+01, 7.5000e+01, 9.5000e+02,
        1.0000e+00, 1.8800e+02, 2.3860e+01, 7.0000e+00, 6.4000e+03])
tensor(0.)


In [4]:
def model_accuracy(model, x, true_y):
    predicted = model.predict(x)
    return (np.sum(predicted == true_y["Player1Wins"].values) / true_y.shape[0])

In [5]:
def evaluate_model(name, model, train_x, train_y, val_x, val_y):
    train_acc = model_accuracy(model, train_x, train_y)
    val_acc = model_accuracy(model, val_x, val_y)
    print(f'{name}: \n\t--- Training accuracy: {train_acc*100 :.2f}%, Testing accuracy: {val_acc*100:.2f}%')
    return

In [11]:
# from math import floor
# def plotSVC(title, X, svc):
#     x_min, x_max = floor(X[:, 0].min() - 1), floor(X[:, 0].max() + 1)
#     y_min, y_max = floor(X[:, 1].min() - 1), floor(X[:, 1].max() + 1)
#     h = 5
#     xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
#     plt.subplot(1, 1, 1)
#     Z = svc.predict(np.c_[xx.ravel(), yy.ravel()])
#     Z = Z.reshape(xx.shape)
#     plt.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.8)
#     plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Paired)
#     plt.xlabel('Sepal length')
#     plt.ylabel('Sepal width')
#     plt.xlim(xx.min(), xx.max())
#     plt.title(title)
#     plt.show()

## SVM with default hyperparameters

In [47]:
from sklearn.svm import SVC

train_x = train.loc[:, train.columns != 'Player1Wins']
train_y = train.loc[:, train.columns == 'Player1Wins']
test_x = test.loc[:, test.columns != 'Player1Wins']
test_y = test.loc[:, test.columns == 'Player1Wins']

# training SVC Classifier with default hyperparameters
svm = SVC(max_iter = 10000).fit(train_x, train_y)

/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [48]:
evaluate_model('SVC with Default Hyperparameters', svm, train_x, train_y, test_x, test_y)

SVC with Default Hyperparameters: 
	--- Training accuracy: 65.21%, Testing accuracy: 65.44%


In [49]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components = 2).fit(train_x)
# pca = pca.transform(train_x)

### Tuning Hyperparameters

In [50]:
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import GridSearchCV

@ignore_warnings(category=ConvergenceWarning)
def do_grid_search(model, param_grid, train_x, train_y):

    m = GridSearchCV(model, param_grid, scoring = 'accuracy')
    return m.fit(train_x, train_y)

In [40]:
# potential parameter grid

param_grid = dict(C = [.001, .01, .1, .5, 1, 5],
                  kernel = ['linear', 'poly', 'rbf', 'sigmoid'])

gs_res = do_grid_search(svm, param_grid, train_x, train_y.values.ravel())

/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: D

/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: D

/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: D

/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: D

/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: D

In [42]:
# best parameters found
gs_res.best_params_  # {'C': 5, 'kernel': 'rbf'}

{'C': 5, 'kernel': 'rbf'}

## Let's train the model and evaluate it with tuned hyperparameters

In [51]:
svm_tuned = SVC(max_iter = 10000).set_params(**gs_res.best_params_).fit(train_x, train_y)

/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/akshaysharma/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [52]:
evaluate_model('SVC with Tuned Hyperparameters', svm_tuned, train_x, train_y, test_x, test_y)

SVC with Tuned Hyperparameters: 
	--- Training accuracy: 65.17%, Testing accuracy: 65.36%
